In [0]:
from google.colab import drive

In [20]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
!wget http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
!tar -xvf vgg_16_2016_08_28.tar.gz
!rm vgg_16_2016_08_28.tar.gz
!ls

--2019-03-02 15:06:27--  http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513324920 (490M) [application/x-tar]
Saving to: ‘vgg_16_2016_08_28.tar.gz’

vgg_16_2016_08_28.t 100%[===================>] 489.54M  64.0MB/s    in 8.4s    

2019-03-02 15:06:35 (58.4 MB/s) - ‘vgg_16_2016_08_28.tar.gz’ saved [513324920/513324920]

vgg_16.ckpt
vgg_16.ckpt


In [0]:
!wget http://cs231n.stanford.edu/coco-animals.zip

--2019-02-26 17:14:45--  http://cs231n.stanford.edu/coco-animals.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181492696 (173M) [application/zip]
Saving to: ‘coco-animals.zip’

coco-animals.zip    100%[===================>] 173.08M  12.4MB/s    in 17s     

2019-02-26 17:15:02 (10.2 MB/s) - ‘coco-animals.zip’ saved [181492696/181492696]



In [0]:
import argparse
import os

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets


In [31]:
!ls '/content/gdrive/My Drive/coco animal dataset/coco-animals/train'

bear  bird  cat  dog  giraffe  horse  sheep  zebra


In [0]:
#VGG Mean
VGG_MEAN = [123.68, 116.78, 103.94]

In [0]:
def list_images(directory):
    """
    Get all the images and labels in directory/label/*.jpg
    """
    labels = os.listdir(directory)
    # Sort the labels so that training and validation get them in the same order
    labels.sort()

    files_and_labels = []
    for label in labels:
        for f in os.listdir(os.path.join(directory, label)):
            files_and_labels.append((os.path.join(directory, label, f), label))

    filenames, labels = zip(*files_and_labels)
    filenames = list(filenames)
    labels = list(labels)
    unique_labels = list(set(labels))

    label_to_int = {}
    for i, label in enumerate(unique_labels):
        label_to_int[label] = i

    labels = [label_to_int[l] for l in labels]

    return filenames, labels

In [0]:
def check_accuracy(sess, correct_prediction, is_training, dataset_init_op):
    """
    Check the accuracy of the model on either train or val (depending on dataset_init_op).
    """
    # Initialize the correct dataset
    sess.run(dataset_init_op)
    num_correct, num_samples = 0, 0
    while True:
        try:
            correct_pred = sess.run(correct_prediction, {is_training: False})
            num_correct += correct_pred.sum()
            num_samples += correct_pred.shape[0]
        except tf.errors.OutOfRangeError:
            break

    # Return the fraction of datapoints that were correctly classified
    acc = float(num_correct) / num_samples
    return acc


In [0]:
def main():
    # Get the list of filenames and corresponding list of labels for training et validation
    train_filenames, train_labels = list_images('/content/gdrive/My Drive/coco animal dataset/coco-animals/train')
    val_filenames, val_labels = list_images('/content/gdrive/My Drive/coco animal dataset/coco-animals/val')
    assert set(train_labels) == set(val_labels),\
           "Train and val labels don't correspond:\n{}\n{}".format(set(train_labels),
                                                                   set(val_labels))

    num_classes = len(set(train_labels))

    #Define computation graph
    graph = tf.Graph()
    with graph.as_default():
        
        
        # Preprocessing (for both training and validation):
        # (1) Decode the image from jpg format
        # (2) Resize the image so its smaller side is 256 pixels long
        def _parse_function(filename, label):
            image_string = tf.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)          # (1)
            image = tf.cast(image_decoded, tf.float32)

            smallest_side = 256.0
            height, width = tf.shape(image)[0], tf.shape(image)[1]
            height = tf.to_float(height)
            width = tf.to_float(width)

            scale = tf.cond(tf.greater(height, width),
                            lambda: smallest_side / width,
                            lambda: smallest_side / height)
            new_height = tf.to_int32(height * scale)
            new_width = tf.to_int32(width * scale)

            resized_image = tf.image.resize_images(image, [new_height, new_width])  # (2)
            return resized_image, label

        # Preprocessing (for training)
        # (3) Take a random 224x224 crop to the scaled image
        # (4) Horizontally flip the image with probability 1/2
        # (5) Substract the per color mean `VGG_MEAN`
        # Note: we don't normalize the data here, as VGG was trained without normalization
        def training_preprocess(image, label):
            crop_image = tf.random_crop(image, [224, 224, 3])                       # (3)
            flip_image = tf.image.random_flip_left_right(crop_image)                # (4)

            means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
            centered_image = flip_image - means                                     # (5)

            return centered_image, label

        # Preprocessing (for validation)
        # (3) Take a central 224x224 crop to the scaled image
        # (4) Substract the per color mean `VGG_MEAN`
        # Note: we don't normalize the data here, as VGG was trained without normalization
        def val_preprocess(image, label):
            crop_image = tf.image.resize_image_with_crop_or_pad(image, 224, 224)    # (3)

            means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
            centered_image = crop_image - means                                     # (4)

            return centered_image, label

        # Training dataset
        train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
        train_dataset = train_dataset.map(_parse_function,
            num_parallel_calls=4)
        train_dataset = train_dataset.map(training_preprocess,
            num_parallel_calls=4)
        train_dataset = train_dataset.shuffle(buffer_size=10000)  # don't forget to shuffle
        batched_train_dataset = train_dataset.batch(32)

        # Validation dataset
        val_dataset = tf.data.Dataset.from_tensor_slices((val_filenames, val_labels))
        val_dataset = val_dataset.map(_parse_function,
            num_parallel_calls=4)
        val_dataset = val_dataset.map(val_preprocess,
            num_parallel_calls=4)
        batched_val_dataset = val_dataset.batch(32)

        #Iterator
        iterator = tf.data.Iterator.from_structure(batched_train_dataset.output_types,
                                                           batched_train_dataset.output_shapes)
        images, labels = iterator.get_next()

        train_init_op = iterator.make_initializer(batched_train_dataset)
        val_init_op = iterator.make_initializer(batched_val_dataset)

        # Indicates whether we are in training or in test mode
        is_training = tf.placeholder(tf.bool)

        #Get pretrained model
        vgg = tf.contrib.slim.nets.vgg
        with slim.arg_scope(vgg.vgg_arg_scope(weight_decay=5e-4)):
            logits, _ = vgg.vgg_16(images, num_classes=num_classes, is_training=is_training,
                                   dropout_keep_prob=0.5)

        # Specify where the model checkpoint is (pretrained weights).
        model_path = '/content/gdrive/My Drive/VGG16/vgg_16.ckpt'
        assert(os.path.isfile(model_path))

        # Restore only the layers up to fc7 (included)
        variables_to_restore = tf.contrib.framework.get_variables_to_restore(exclude=['vgg_16/fc8'])
        init_fn = tf.contrib.framework.assign_from_checkpoint_fn(model_path, variables_to_restore)

        # Initialization operation from scratch for the new "fc8" layers
        fc8_variables = tf.contrib.framework.get_variables('vgg_16/fc8')
        fc8_init = tf.variables_initializer(fc8_variables)

        tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        loss = tf.losses.get_total_loss()

        fc8_optimizer = tf.train.GradientDescentOptimizer(1e-3)
        fc8_train_op = fc8_optimizer.minimize(loss, var_list=fc8_variables)

        full_optimizer = tf.train.GradientDescentOptimizer(1e-5)
        full_train_op = full_optimizer.minimize(loss)

        # Evaluation metrics
        prediction = tf.to_int32(tf.argmax(logits, 1))
        correct_prediction = tf.equal(prediction, labels)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        tf.get_default_graph().finalize()

    
    
    with tf.Session(graph=graph) as sess:
        init_fn(sess)  # load the pretrained weights
        sess.run(fc8_init)  # initialize the new fc8 layer
        

        # Update only the last layer for a few epochs.
        for epoch in range(10):
            # Run an epoch over the training data.
            print('Starting epoch %d / %d' % (epoch + 1, 10))
            sess.run(train_init_op)
            while True:
                try:
                    _ = sess.run(fc8_train_op, {is_training: True})
                except tf.errors.OutOfRangeError:
                    break

            # Check accuracy on the train and val sets every epoch.
            train_acc = check_accuracy(sess, correct_prediction, is_training, train_init_op)
            val_acc = check_accuracy(sess, correct_prediction, is_training, val_init_op)
            print('Train accuracy: %f' % train_acc)
            print('Val accuracy: %f\n' % val_acc)


        # Train the entire model for a few more epochs, continuing with the *same* weights.
        for epoch in range(10):
            print('Starting epoch %d / %d' % (epoch + 1,10))
            sess.run(train_init_op)
            while True:
                try:
                    _ = sess.run(full_train_op, {is_training: True})
                except tf.errors.OutOfRangeError:
                    break

            # Check accuracy on the train and val sets every epoch
            train_acc = check_accuracy(sess, correct_prediction, is_training, train_init_op)
            val_acc = check_accuracy(sess, correct_prediction, is_training, val_init_op)
            print('Train accuracy: %f' % train_acc)
            print('Val accuracy: %f\n' % val_acc)
         
        saver.save(sess, '/content/gdrive/My Drive/VGG16/coco_vgg16')
        print("Model Saved")

In [10]:
if __name__ == '__main__':
#     args = parser.parse_args()
    main()

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/VGG16/vgg_16.ckpt
Starting epoch 1 / 10
Train accuracy: 0.575000
Val accuracy: 0.595000

Starting epoch 2 / 10
Train accuracy: 0.736250
Val accuracy: 0.725000

Starting epoch 3 / 10
Train accuracy: 0.776250
Val accuracy: 0.765000

Starting epoch 4 / 10
Train accuracy: 0.806250
Val accuracy: 0.800000

Starting epoch 5 / 10
Train accuracy: 0.812500
Val accuracy: 0.790000

Starting epoch 6 / 10
Train accuracy: 0.833750
Val accuracy: 0.815000

Starting epoch 7 / 10
Train accuracy: 0.845000
Val accuracy: 0.820000

Starting epoch 8 / 10
Train accuracy: 0.842500
V

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0
